In [1]:
!vessl --version

vessl, version 0.1.192


In [6]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 13.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
print(os.path.exists(r"E:\2025_Jeju_SectorCoupling\PVgen_Jeju"))  # True여야 함
print(os.path.exists(r"F:\Junkyo\2025 Jeju Sector Coupling\PVgen"))  # True여야 함


False
True


In [9]:
import os
print(os.getcwd())  # 현재 작업 디렉토리 확인


/My Drive


In [11]:
print(os.path.exists("E:/"))
subst


False


NameError: name 'subst' is not defined

In [7]:
import os
import re
import numpy as np
import rasterio
import xml.etree.ElementTree as ET

# 입력 및 출력 디렉토리 설정
dirs = {k: fr"F:\Junkyo\2025 Jeju Sector Coupling\PVgen_{k.capitalize()}" for k in ["jeju", "gosan", "seoguipo"]}
output_dir = r"F:\Junkyo\2025 Jeju Sector Coupling\PVgen"
os.makedirs(output_dir, exist_ok=True)

# 래스터 파일 정렬
files = {k: sorted(f for f in os.listdir(v) if f.endswith(".tif")) for k, v in dirs.items()}
if len(set(map(len, files.values()))) != 1:
    raise ValueError("각 디렉토리에 동일한 개수의 래스터 파일이 있어야 합니다.")

# 파일명 정리 함수
def clean_filename(filename):
    return re.sub(r"^.*?(\d{4}-\d{2}-\d{2}.*)", r"\1", filename)

# TFW 및 XML 저장 함수
def save_tfw(transform, path):
    with open(path, 'w') as f:
        f.writelines(f"{v:.10f}\n" for v in [transform.a, transform.b, transform.d, transform.e, transform.xoff, transform.yoff])

def save_xml(metadata, path):
    root = ET.Element("RasterMetadata")
    ET.SubElement(root, "CRS").text = metadata["crs"]
    ET.SubElement(root, "NoDataValue").text = str(metadata["nodata"])
    transform = ET.SubElement(root, "Transform")
    for k, v in zip(["PixelSizeX", "PixelSizeY", "UpperLeftX", "UpperLeftY"], metadata["transform"][:6:2]):
        ET.SubElement(transform, k).text = str(v)
    sources = ET.SubElement(root, "SourceFiles")
    [ET.SubElement(sources, "File").text for f in metadata["source_files"]]
    ET.ElementTree(root).write(path, encoding="utf-8", xml_declaration=True)

# 래스터 병합 및 저장
for f1, f2, f3 in zip(*files.values()):
    raster_paths = {k: os.path.join(dirs[k], f) for k, f in zip(dirs, [f1, f2, f3])}
    with rasterio.open(raster_paths["jeju"]) as ref:
        transform, profile = ref.transform, ref.profile.copy()
        arrays = [np.nan_to_num(rasterio.open(r).read(1), nan=0) for r in raster_paths.values()]
        result_arr = sum(arrays)
    
    output_name = f"TotalSolargen_{clean_filename(f1)}"
    output_paths = {ext: os.path.join(output_dir, output_name + ext) for ext in [".tif", ".tfw", ".xml"]}
    
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open(output_paths[".tif"], 'w', **profile) as dst:
        dst.write(result_arr, 1)
    
    save_tfw(transform, output_paths[".tfw"])
    save_xml({"source_files": [f1, f2, f3], "transform": transform.to_gdal(), "crs": str(ref.crs), "nodata": profile.get("nodata", None)}, output_paths[".xml"])
    
    print(f"저장 완료: {output_paths}")

print("모든 래스터 병합 완료.")

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\2025_Jeju_SectorCoupling\\PVgen_Jeju'